In [0]:
# 출처: https://github.com/2alive3s/Fake_news

import tensorflow as tf
import numpy as np

#from tensorflow.contrib.rnn import stack_bidirectional_rnn as bi_rnn


class Affine(object):
# Combine all the pooled features
    def __init__( # 초기화 수행
      self, sequence_length_head, sequence_length_body, num_classes, vocab_size_head, vocab_size_body, 
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.1):
    # sequence_length_head = 기사 제목의 길이
    # sequence_length_body = 기사 본문의 길이
    # num_classes = 클래스 개수
    # vocab_size_head = 제목의 단어 개수
    # vocab_size_body = 본문의 단어 개수
    # embedding_size = 임베딩 크기
    # filter_sizes = 필터 크기
    # num_filters = 필터 개수
    # l2_reg_lambda = learning_rate = 0.1
        
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        self.input_x_head = tf.placeholder(tf.int32, [None, sequence_length_head], name="input_x_head")
        self.input_x_body = tf.placeholder(tf.int32, [None, sequence_length_body], name="input_x_body") # 플레이스홀더 연결
        
        # Embedding layer
        self.embeddings_head = tf.Variable( #head
                tf.random_uniform([vocab_size_head, embedding_size], -1.0, 1.0),trainable=False) # 정규분포를 따르는 -1~1사이의 난수[단어수,임베딩 크기]
        self.embedded_chars_head = tf.nn.embedding_lookup(self.embeddings_head, self.input_x_head) # embeddings_head에서 input_x_head를 인덱스로 하는 원소 lookup
        self.embedded_chars_expanded_head = tf.expand_dims(self.embedded_chars_head, -1) # embedded_chars_head의 마지막 차원을 추가

        self.embeddings_body = tf.Variable( #body
                tf.random_uniform([vocab_size_body, embedding_size], -1.0, 1.0),trainable=False) # 정규분포를 따르는 -1~1사이의 난수[단어수,임베딩 크기]
        self.embedded_chars_body = tf.nn.embedding_lookup(self.embeddings_body, self.input_x_body) # embeddings_body에서 input_x_body를 인덱스로 하는 원소 lookup
        self.embedded_chars_expanded_body = tf.expand_dims(self.embedded_chars_body, -1) # embedded_chars_body의 마지막 차원을 추가
        
        self.pooled_outputs_head = [] #pooled_outputs_head배열 생성 
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-head-%s" % filter_size): # 범위를 나누어 줌(보기에 편리)
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, 256] # 필터 shape 설정 
                W_head = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W_head") # 제목의 가중치(잘린정규분포를 따름, 표준편차 0.1)
                b_head = tf.Variable(tf.constant(0.1, shape=[256]), name="b_head") # 제목의 편향 설정
                conv_head = tf.nn.conv2d( # 합성곱계층
                    self.embedded_chars_expanded_head,
                    W_head,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                self.h_head = tf.nn.relu(tf.nn.bias_add(conv_head, b_head), name="relu_head") # 편향을 더해주고 활성화함수로 ReLU 적용

        self.pooled_outputs_body = [] # pooled_outputs_body 배열 생성
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-body-%s" % filter_size):# 범위를 나누어 줌(보기에 편리)
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, 1024] # 필터 크기 설정, 이 때, head보다 body가 텍스트 수가 많기 때문에 필터 개수를 더 많이(256<1024) 
                W_body = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W_body") # 본문의 가중치(잘린정규분포를 따름, 표준편차 0.1)
                b_body = tf.Variable(tf.constant(0.1, shape=[1024]), name="b_body") # 본문의 편향 설정
                conv_body = tf.nn.conv2d( # 합성곱계층
                    self.embedded_chars_expanded_body,
                    W_body,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                self.h_body = tf.nn.relu(tf.nn.bias_add(conv_body, b_body), name="relu_body") # 편향을 더해주고 활성화함수로 ReLU 적용
        
        # Maxpooling over the outputs
        l2_loss = tf.constant(0.0) # l2_loss를 0으로 설정
        self.num_filters_total = num_filters * len(filter_sizes) # 총 필터 수 = 필터수 * filter_sizes의 크기
        self.U = tf.Variable(tf.truncated_normal(shape = [256,1024],stddev = 0.01,name = 'U')) # 잘린정규분포를 만족하는 크기가 [256,1024]이고 표준편차가 0.01인 U 
        self.pooled_outputs_head, self.pooled_outputs_body = self.attentive_pooling(self.h_head,self.h_body,sequence_length_head,sequence_length_body)
        # attetive_pooling 함수에 h_head,h_body,sequence_length_head,sequence_length_body를 인자로 넣고 return 값을 pooled_outputs_head,pooled_outputs_body에 저장
        self.sims = self.interact(self.pooled_outputs_head, self.pooled_outputs_body)
        # interact함수에 pooled_outputs_head,pooled_outputs_body를 인자로 넣고 return 값을 sims에 저장
        pooled_outputs = tf.concat([self.pooled_outputs_head,self.sims,self.pooled_outputs_body],-1,name='preconcat') # 데이터를 마지막 차원에 따라 연결
        
        self.h_pool = tf.concat(pooled_outputs, 3, name='concat') # 3차원을 따라 pooled_outputs의 데이터 연결
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, self.num_filters_total+1]) # tf.reshape을 사용하여 h_pool을 일자로 펴기
       
       	#완전연결계층
        W_fc1 = tf.Variable(tf.truncated_normal([1281,1024],stddev=0.1),name="W_fc1") # 완전연결계층의 가중치 설정(잘린정규분포를 따르는 크기가 [1281,1024]이고 표준편차=0.1)
        b_fc1 = tf.Variable(tf.constant(0.1,shape=[1024]),name="b_fc1") # 완전연결계층의 편향 설정
        h_fc1 = tf.nn.relu(tf.matmul(self.h_pool_flat,W_fc1) + b_fc1) # 가중치*h_pool_flat +편향을 해주고 활성화함수 ReLU를 적용 
            
        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(h_fc1, self.dropout_keep_prob) #dropout 적용, keep_prob : 드롭아웃 되지 않을 확률

        # Final (unnormalized) scores and predictions 
        with tf.name_scope("output"):
            self.W = tf.get_variable(
                "W",
                shape=[1024, num_classes],
                initializer=tf.contrib.layers.xavier_initializer()) # 가중치 설정
            self.b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b") # 편향 설정
            l2_loss += tf.nn.l2_loss(self.W) # W의 l2_loss 구해서 l2_loss에 더하기
            l2_loss += tf.nn.l2_loss(self.b) # b의 l2_loss 구해서 l2_loss에 더하기
            self.scores = tf.nn.xw_plus_b(self.h_drop, self.W, self.b, name="scores") # scores = h_drop*W+b
            self.probabilities = tf.nn.softmax(self.scores) # softmax함수 적용하여 확률 얻기
            self.predictions = tf.argmax(self.scores, 1, name="predictions") # scores가 가장 큰 데이터를 predictions에 저장

        # CalculateMean cross-entropy loss
        with tf.name_scope("loss"):
            print(self.scores.shape)
            losses = tf.nn.softmax_cross_entropy_with_logits(logits = self.scores, labels = self.input_y) # cross_entropy loss 구하기
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss # loss 최소화

        # Accuracy
        with tf.name_scope("accuracy"):
            print("%d/%d",self.predictions,self.input_y)
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1)) # 예측한 것과 정답이 같은지 확인
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy") # tf.cast를 사용하여 정확도 측정

    def attentive_pooling(self,input_left,input_right,sequence_length_head,sequence_length_body):
        
        head = tf.reshape(input_left,[-1,sequence_length_head-3+1,256],name = 'Q') # head에 input_left를 reshape하여 저장
        body = tf.reshape(input_right,[-1,sequence_length_body-3+1,1024],name = 'A') # body에 input_right를 reshape하여 저장
        # G = tf.tanh(tf.matmul(tf.matmul(Q,self.U),\
        # A,transpose_b = True),name = 'G')
        print("head",head.shape)
        print("U",self.U.shape)
        first = tf.matmul(tf.reshape(head,[-1,256]),self.U) # U와 head 곱하기
        print("first",first.shape)
        second_step = tf.reshape(first,[-1,sequence_length_head - 3 + 1,1024]) # U와 head곱한 것을 reshape
        print("second_step",second_step.shape)
        result = tf.matmul(second_step,tf.transpose(body,perm = [0,2,1])) # head*U*body의 transpose
        print("resultshape",result.shape)
        # print 'result',result
        G = tf.tanh(result) # result에 tanh
        
        # G = result
        # column-wise pooling ,row-wise pooling
        row_pooling = tf.reduce_max(G,1,True,name = 'row_pooling') # 1차원에 따라 최댓값 선택_row
        col_pooling = tf.reduce_max(G,2,True,name = 'col_pooling') # 2차원에 따라 최댓값 선택_col
    
        self.attention_q = tf.nn.softmax(col_pooling,1,name = 'attention_q') # attention_q에 col_pooling을 1차원에서 softmax 함수를 적용한 값 저장
        print(self.attention_q)
        self.see = self.attention_q

        self.attention_a = tf.nn.softmax(row_pooling,name = 'attention_a') # attention_a에 row_pooling을 마지막 차원에서 softmax 함수를 적용한 값 저장
        print(self.attention_a)
        R_q = tf.reshape(tf.matmul(head,self.attention_q,transpose_a = 1),[-1,256],name = 'R_q') #head와 attention_q를 곱해서 reshape([-1,256])
        R_a = tf.reshape(tf.matmul(self.attention_a,body),[-1,1024],name = 'R_a') # attention_a와 body를 곱해서 reshape([-1,1024])
        print(R_q)
        print(R_a)

        return R_q,R_a 
    
    def interact(self, head, body):
        # Compute similarity
        with tf.name_scope("similarity"):
            W = tf.get_variable(
                "W_sim",
                shape=[256, 1024],
                initializer=tf.contrib.layers.xavier_initializer()) # 가중치 얻기
            # print 'q_pooling',self.q_pooling
            # print 'num_filters',self.num_filters_total
            self.transform_head = tf.matmul(head, W) # head와 W곱하기
            print(self.transform_head)
            sims = tf.reduce_sum(tf.multiply(self.transform_head, self.body), 1, keep_dims=True) #transform_head와 body를 곱한 것을 1차원에 따라 원소들을 더함
            
        return sims #sims return